In [1]:
import numpy as np
import pandas as pd
import math
import pickle
import scipy.sparse as sp
from sklearn.preprocessing import LabelEncoder
import numpy as np
import matplotlib.pyplot as plt
import os
from sklearn.preprocessing import LabelEncoder
from scipy.sparse import csr_matrix, hstack
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss

In [2]:
# train/test split
f = open('../data/full_train_yx','rb')
train = pickle.load(f)
n_train = len(train)

f = open('../data/full_test_yx','rb')
test = pickle.load(f)
n_test = len(test)

In [3]:
# load data
datadir = '../data/'
data1 = pd.read_csv(os.path.join(datadir,'gender_age_train.csv'),
                      index_col='device_id')

data2 = pd.read_csv(os.path.join(datadir,'gender_age_test.csv'),
                     index_col = 'device_id')

data = pd.concat([data1, data2],sort=False)

gatrain = data.loc[train]
gatest = data.loc[test]

In [4]:
# get app data, brand/device info
gatrain['trainrow'] = np.arange(gatrain.shape[0])
gatest['testrow'] = np.arange(gatest.shape[0])
phone = pd.read_csv(os.path.join(datadir,'phone_brand_device_model.csv'))
# Get rid of duplicate device ids in phone
phone = phone.drop_duplicates('device_id',keep='first').set_index('device_id')
events = pd.read_csv(os.path.join(datadir,'events.csv'),
                     parse_dates=['timestamp'], index_col='event_id')
appevents = pd.read_csv(os.path.join(datadir,'app_events.csv'), 
                        usecols=['event_id','app_id','is_active'],
                        dtype={'is_active':bool})
applabels = pd.read_csv(os.path.join(datadir,'app_labels.csv'))
appencoder = LabelEncoder().fit(appevents.app_id)
appevents['app'] = appencoder.transform(appevents.app_id)
napps = len(appencoder.classes_)
deviceapps = (appevents.merge(events[['device_id']], how='left',left_on='event_id',right_index=True)
                       .groupby(['device_id','app'])['app'].agg(['size'])
                       .merge(gatrain[['trainrow']], how='left', left_index=True, right_index=True)
                       .merge(gatest[['testrow']], how='left', left_index=True, right_index=True)
                       .reset_index())
deviceapps.head()



/Users/meganwilliams/p3/lib/python3.7/site-packages/numpy/lib/arraysetops.py:571: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,device_id,app,size,trainrow,testrow
0,-9222956879900151005,548,18,49974.0,NaN
1,-9222956879900151005,1096,18,49974.0,NaN
2,-9222956879900151005,1248,26,49974.0,NaN
3,-9222956879900151005,1545,12,49974.0,NaN
4,-9222956879900151005,1664,18,49974.0,NaN


In [16]:
d = deviceapps.fillna(0)
user = d.trainrow + (d.testrow != 0) * ( 59714 + d.testrow)
d['user'] = user
d.user = d.user.astype(int)
d.head()

,device_id,app,size,trainrow,testrow,user
0,-9222956879900151005,548,18,49974.0,0.0,49974
1,-9222956879900151005,1096,18,49974.0,0.0,49974
2,-9222956879900151005,1248,26,49974.0,0.0,49974
3,-9222956879900151005,1545,12,49974.0,0.0,49974
4,-9222956879900151005,1664,18,49974.0,0.0,49974


In [17]:
# create adjacency matrix for user/app data
adj_matrix = d[['user', 'app','size']].copy()
adj_matrix.head()

,user,app,size
0,49974,548,18
1,49974,1096,18
2,49974,1248,26
3,49974,1545,12
4,49974,1664,18


In [18]:
# give app nodes unique id
print(max(adj_matrix['user']))
to_add = max(adj_matrix['user'])
adj_matrix.app = adj_matrix.app.add(to_add)
adj_matrix.head()

74640


,user,app,size
0,49974,75188,18
1,49974,75736,18
2,49974,75888,26
3,49974,76185,12
4,49974,76304,18


In [22]:
# save adjacency matrix
N = max(adj_matrix.app) + 1
A = adj_matrix.values
B = sp.csc_matrix((A[:,2],(A[:,0],A[:,1])),shape=(N,N))
C = B + B.T
print(C.shape)
sc.io.savemat('data.mat', mdict={'arr': C})

(93877, 93877)


In [23]:
# write matrix to file
data = adj_matrix.to_csv(sep=' ', columns=['user','app','size'],index=False,header=False)
f = open("adj.txt", "w+")
f.write(data)
f.close()

In [74]:
# load embeddings
d = 64
col_names = [str(i) for i in range(d)]
col_names = ['trainrow'] + col_names 
df = pd.read_csv("data.embeddings", names=col_names,header=None,sep=' ')
df = df.set_index('trainrow')
df = df.sort_index()
df.head()


,0,1,2,3,4,5,6,7,8,9,...,54,55,56,57,58,59,60,61,62,63
trainrow,,,,,,,,,,,,,,,,,,,,,
0,-0.245789,0.665561,0.180012,0.596770,-0.594188,0.076157,0.555067,-0.118514,-0.112164,-0.121403,...,0.032931,0.156127,-0.453464,0.113413,0.348636,0.204719,0.488032,-0.226228,0.042174,-0.048134
1,0.557043,0.650613,0.405881,-0.038715,0.933904,-0.755581,-0.337868,0.063155,0.307639,0.037800,...,-0.111880,0.586298,-0.186547,-0.388236,0.222154,-0.477946,0.404360,0.448119,-0.203529,-0.732918
2,-0.899203,-0.276360,0.386593,0.039753,0.795235,0.108771,0.232204,-0.145139,-0.072643,-0.204576,...,-0.504175,-0.157142,-0.084664,-0.450989,-0.121691,-0.353925,0.394439,0.277881,0.050928,-0.462906
4,-0.091535,0.586725,-0.073254,-0.156283,0.384207,-0.179355,0.032990,0.736327,-0.680608,-0.267607,...,-0.005289,0.443636,0.213787,-0.062880,0.482620,0.362344,0.020619,0.583790,-0.254955,-0.153782
5,0.525404,-0.408179,0.328601,0.113633,0.265358,0.637055,-0.193792,0.150604,-0.282524,-0.362874,...,0.904497,0.051784,0.123839,-0.416896,0.257664,-0.492158,0.031570,0.402654,0.281279,-0.520026


In [75]:
# fill missing values
indices = np.arange(74645)
df = df.reindex(index=indices, fill_value=0)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,54,55,56,57,58,59,60,61,62,63
trainrow,,,,,,,,,,,,,,,,,,,,,
0,-0.245789,0.665561,0.180012,0.596770,-0.594188,0.076157,0.555067,-0.118514,-0.112164,-0.121403,...,0.032931,0.156127,-0.453464,0.113413,0.348636,0.204719,0.488032,-0.226228,0.042174,-0.048134
1,0.557043,0.650613,0.405881,-0.038715,0.933904,-0.755581,-0.337868,0.063155,0.307639,0.037800,...,-0.111880,0.586298,-0.186547,-0.388236,0.222154,-0.477946,0.404360,0.448119,-0.203529,-0.732918
2,-0.899203,-0.276360,0.386593,0.039753,0.795235,0.108771,0.232204,-0.145139,-0.072643,-0.204576,...,-0.504175,-0.157142,-0.084664,-0.450989,-0.121691,-0.353925,0.394439,0.277881,0.050928,-0.462906
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,-0.091535,0.586725,-0.073254,-0.156283,0.384207,-0.179355,0.032990,0.736327,-0.680608,-0.267607,...,-0.005289,0.443636,0.213787,-0.062880,0.482620,0.362344,0.020619,0.583790,-0.254955,-0.153782


In [76]:
# Split train/test embeddings
X = df.as_matrix()
Xtr = X[0:n_train,:]
Xte = X[n_train:,:]
print(Xtr.shape)

(59716, 64)


/Users/meganwilliams/p3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


In [29]:
# construct brand feature vector
brandencoder = LabelEncoder().fit(phone.phone_brand)
phone['brand'] = brandencoder.transform(phone['phone_brand'])
gatrain['brand'] = phone['brand']
gatest['brand'] = phone['brand']
Xtr_brand = csr_matrix((np.ones(gatrain.shape[0]), 
                       (gatrain.trainrow, gatrain.brand)))
Xte_brand = csr_matrix((np.ones(gatest.shape[0]), 
                       (gatest.testrow, gatest.brand)))
print('Brand features: train shape {}, test shape {}'.format(Xtr_brand.shape, Xte_brand.shape))

Brand features: train shape (59716, 131), test shape (14929, 131)


In [30]:
# construct app feature vector
d = deviceapps.dropna(subset=['trainrow'])
Xtr_app = csr_matrix((np.ones(d.shape[0]), (d.trainrow, d.app)), 
                      shape=(gatrain.shape[0],napps))
d = deviceapps.dropna(subset=['testrow'])
Xte_app = csr_matrix((np.ones(d.shape[0]), (d.testrow, d.app)), 
                      shape=(gatest.shape[0],napps))
print('Apps data: train shape {}, test shape {}'.format(Xtr_app.shape, Xte_app.shape))

Apps data: train shape (59716, 19237), test shape (14929, 19237)


In [31]:
# construct app label feature vector
applabels = applabels.loc[applabels.app_id.isin(appevents.app_id.unique())]
applabels['app'] = appencoder.transform(applabels.app_id)
labelencoder = LabelEncoder().fit(applabels.label_id)
applabels['label'] = labelencoder.transform(applabels.label_id)
nlabels = len(labelencoder.classes_)
devicelabels = (deviceapps[['device_id','app']]
                .merge(applabels[['app','label']])
                .groupby(['device_id','label'])['app'].agg(['size'])
                .merge(gatrain[['trainrow']], how='left', left_index=True, right_index=True)
                .merge(gatest[['testrow']], how='left', left_index=True, right_index=True)
                .reset_index())
devicelabels.head()

d = devicelabels.dropna(subset=['trainrow'])
Xtr_label = csr_matrix((np.ones(d.shape[0]), (d.trainrow, d.label)), 
                      shape=(gatrain.shape[0],nlabels))
d = devicelabels.dropna(subset=['testrow'])
Xte_label = csr_matrix((np.ones(d.shape[0]), (d.testrow, d.label)), 
                      shape=(gatest.shape[0],nlabels))
print('Labels data: train shape {}, test shape {}'.format(Xtr_label.shape, Xte_label.shape))

Labels data: train shape (59716, 492), test shape (14929, 492)


In [32]:
# construct model feature vector
m = phone.phone_brand.str.cat(phone.device_model)
modelencoder = LabelEncoder().fit(m)
phone['model'] = modelencoder.transform(m)
gatrain['model'] = phone['model']
gatest['model'] = phone['model']
Xtr_model = csr_matrix((np.ones(gatrain.shape[0]), 
                       (gatrain.trainrow, gatrain.model)))
Xte_model = csr_matrix((np.ones(gatest.shape[0]), 
                       (gatest.testrow, gatest.model)))
print('Model features: train shape {}, test shape {}'.format(Xtr_model.shape, Xte_model.shape))

Model features: train shape (59716, 1667), test shape (14929, 1667)


In [33]:
# get labels
labels = gatrain[['trainrow','group']].copy()
labels.group = labels.group.astype('category')
labels.group = labels.group.cat.codes
labels.head()

,trainrow,group
device_id,,
3939996495880662904,0,8
-5127195038907890249,1,0
3074841628121636464,2,10
1930589242199599785,3,10
-2845082486521958712,4,10


In [34]:
# encode labels as number 
targetencoder = LabelEncoder().fit(gatrain.group)
y = targetencoder.transform(gatrain.group)
nclasses = len(targetencoder.classes_)

In [77]:
# concatenate feature vectors
Xtrain = hstack((Xtr_brand, Xtr_model, Xtr_app, Xtr_label, Xtr), format='csr')
Xtest =  hstack((Xte_brand, Xte_model, Xte_app, Xte_label, Xte), format='csr')
print('All features: train shape {}, test shape {}'.format(Xtrain.shape, Xtest.shape))

All features: train shape (59716, 21591), test shape (14929, 21591)


In [36]:
from sklearn.model_selection import StratifiedKFold

# cross validation
def score(clf,n=1, random_state = 0):
    kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=random_state)
    pred = np.zeros((y.shape[0],nclasses))
    avg_acc = 0
    reg_acc = 0
    for itrain, itest in kf.split(Xtrain, y):
        Xtr, Xte = Xtrain[itrain, :], Xtrain[itest, :]
        ytr, yte = y[itrain], y[itest]
        clf.fit(Xtr, ytr)
        pred[itest,:] = clf.predict_proba(Xte)
        reg_acc += clf.score(Xte,yte)
        best_n = np.argsort(pred[itest,:], axis=1)[:,-n:] # top 3
        if n==1:
            acc = clf.score(Xte,yte)
        else:
            acc = 0
            m = best_n.shape[0]
            for i in range(m):
                if yte[i] in list(best_n[i,:]):
                    acc += 1
            acc = acc / np.float(m)
        avg_acc += acc
    avg_acc = avg_acc/np.float(5)
    reg_acc = reg_acc/np.float(5)
    return (reg_acc, avg_acc), log_loss(y, pred)

In [37]:
# cross-validate
Cs = [0.0001, 0.001,0.01,0.1,1]
for c in Cs:
    print(c)
    clf = LogisticRegression(C=c,multi_class='multinomial',solver='newton-cg',max_iter=1000)
    acc, loss = score(clf,3)
    print("Accuracy:",acc[0])
    print("Accuracy@3",acc[1])

0.0001


/Users/meganwilliams/p3/lib/python3.7/site-packages/scipy/optimize/linesearch.py:313: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/Users/meganwilliams/p3/lib/python3.7/site-packages/sklearn/utils/optimize.py:195: UserWarning: Line Search failed
  warnings.warn('Line Search failed')
/Users/meganwilliams/p3/lib/python3.7/site-packages/scipy/optimize/linesearch.py:313: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/Users/meganwilliams/p3/lib/python3.7/site-packages/sklearn/utils/optimize.py:195: UserWarning: Line Search failed
  warnings.warn('Line Search failed')


Accuracy: 0.15116621142642872
Accuracy@3 0.3993901845342712
0.001
Accuracy: 0.17824457005416888
Accuracy@3 0.4339543974579837
0.01
Accuracy: 0.19453848644419622
Accuracy@3 0.4589567108742208
0.1
Accuracy: 0.19338217343989195
Accuracy@3 0.4562773462633201
1
Accuracy: 0.1820452003068944
Accuracy@3 0.4383586055279455


In [78]:
# Test
targetencoder = LabelEncoder().fit(gatest.group)
y_test = targetencoder.transform(gatest.group)

c = 0.01
clf = LogisticRegression(C=c,multi_class='multinomial',solver='newton-cg',max_iter=1000).fit(Xtrain,y)
acc = clf.score(Xtest,y_test)
print("Accuracy", acc)

n = 3 # top 3
pred= clf.predict_proba(Xtest)
best_n = np.argsort(pred, axis=1)[:,-n:] # top 3
acc = 0
m = best_n.shape[0]
for i in range(m):
    if y_test[i] in list(best_n[i,:]):
        acc += 1
acc = acc / np.float(m)
print("Accuracy@3", acc)

Accuracy 0.1955254873065845
Accuracy@3 0.46044611159488247
